In [ ]:
# Imports básicos
import numpy as np
from numpy import random
import pandas as pd
import math
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')
random.RandomState(1)

# Imports scikit-learn
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import make_blobs, make_moons
from sklearn.metrics import accuracy_score

# Imports pytroch
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.utils.data as data_utils
import torchvision
from torchvision import datasets
from torchvision import transforms
import torchvision.transforms.functional as TF
from PIL import Image as Pil_Image

# Imports para plots
import matplotlib.pyplot as plt
%matplotlib inline

<img src="https://www.ifsc.edu.br/image/layout_set_logo?img_id=1319584&t=1602803233260" width="20%">

<center>

---
# **Introdução a Redes Neurais para Classificação**
## Implementação em Python com Pytorch (Parte 3)
## <u>Prof. Carlos Andres Ferrero</u>
## Instituto Federal de Santa Catarina (IFSC), Câmpus Lages
## Grupo de Pesquisa em Análise Inteligente de Dados (IDA-IFSC)
## *Semana Nacional de Ciência e Tecnologia (SNCT/IFSC)*
---
</center>

# Aplicações em Classificação de Imagens

- Construir Redes Neurais para dados que envolvam Imagens é uma das aplicações mais comuns nos dias atuais
- Essa análise envolve geralmente os tipos de camada:
  - Convolutional Layers
  - MaxPool Layers
  - Dropout Layer
  - Dense Layers
- Quando uma rede neural tem várias camadas é chamada de **Deep Neural Network** e o aprendizado chamado de **Deep Learning**.

# Problema de Classificação de Imagens

Neste contexto de pandemia do Covid-19 é de interesse de gestores de estabelecimentos públicos e privados o controle das pessoas que entram e saem dos mesmos, em relação ao uso de máscaras.

O **objetivo** consiste em construir uma rede neural que seja capaz de **classificar imagens de pessoas** nas classes **com máscara** e **sem máscara**.

# 0 - Entender o Problema de Classificação e os Dados Disponíveis

## Baixando e carregando o conjunto de dados

Os dados foram extraídos do seguinte conjunto de dados na plataforma Kaggle: https://www.kaggle.com/prithwirajmitra/covid-face-mask-detection-dataset

Esses dados foram baixados e posicionados em um link do Dropbox para facilitar baixar e utilizar as imagens.


In [ ]:
# Criar pasta para armazenar os dados
!mkdir -p covid-face-mask-detection-dataset
# Baixar e descompactar dados de treino
!wget -q https://www.dropbox.com/s/tvc6os8fvwxifed/Train.zip -O covid-face-mask-detection-dataset/Train.zip
!unzip -o -q covid-face-mask-detection-dataset/Train.zip -d covid-face-mask-detection-dataset/
# Baixar e descompactar dados de teste
!wget -q https://www.dropbox.com/s/r6xoz38nszc6f5k/Test.zip -O covid-face-mask-detection-dataset/Test.zip
!unzip -o -q covid-face-mask-detection-dataset/Test.zip -d covid-face-mask-detection-dataset/

**Transformadores de Imagem** são usados para transformar conjuntos de dados de imagem de forma padronizada. Essas transformações também podem usar informações aleatórias para inserir variabilidade durante a etapa treinamento do modelo (não usaremos isso nesta oportunidade).

In [ ]:
height = 32
width = 32
transform = transforms.Compose([                                
                                transforms.Resize((height,width)), # Redimensionar para 32x32 pixels
                                transforms.ToTensor() # Transformar para o formato adequado de análise
                                ])

train_data=datasets.ImageFolder('covid-face-mask-detection-dataset/Train', transform=transform)
test_data=datasets.ImageFolder('covid-face-mask-detection-dataset/Test', transform=transform)

**Data Loaders** são usados para obter subconjuntos de imagens tamanho pré-definido (*batch_size*) escolhidos aleatoriamente.

In [ ]:
batch_size = 16
train_loader = data_utils.DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(test_data, batch_size=batch_size, shuffle=False)

**Iteradores** são usados para acessar subconjuntos de imagens de *Data Loaders*

In [ ]:
train_iter = iter(train_loader)

In [ ]:
images, labels = next(train_iter)
print('Shape de images:', images.shape)
print('Shape de labels:', labels.shape)

No shape de images o primeiro valor indica o número de imagens do subconjunto (batch), o segundo valor o número de canais (1 em GrayScale e 3 em RGB, por exemplo) e os últimos dois valores são as dimensões da imagem.

No shape de labels o valor indica o número de labels, um para cada imagem.

**Mostrar uma imagem**

Ao mostrar uma imagem usando a biblioteca *matplotlib* temos que transformar as dimensões da imagem, posicionando o número de canais como a última dimensão, por ex., de \[3, 32, 32\] para \[32, 32, 3\].

Classes:
- Classe 0 - Com máscara
- Classe 1 - Sem máscara

In [ ]:
label_desc = {0 : 'com máscara', 1 : 'sem máscara'}

In [ ]:
i = 0
plt.imshow(images[i].permute(1, 2, 0).squeeze())
plt.title('Classe %i - %s' % (labels[i], label_desc[ labels.detach().numpy()[i] ] ))
plt.show()

**Mostrar várias imagens**

In [ ]:
fig = plt.figure(figsize=(24, 7))
rows = 2
columns = math.ceil(batch_size / rows)

for i in range(0, columns*rows):
    img = images[i].permute(1, 2, 0).squeeze()
    fig.add_subplot(rows, columns, i+1, title = 'Classe %i - %s' % (labels[i], label_desc[ labels.detach().numpy()[i] ] ) )
    plt.imshow(img)

# 1 - Definição de Arquitetura

**Representação Visual**

<div align="center">
<img  src="https://raw.githubusercontent.com/anfer86/teaching-datascience-lessons/master/aula08/img/cnn_model.svg">
</div>

- [Link GIF Convolution](https://miro.medium.com/max/494/1*nYf_cUIHFEWU1JXGwnz-Ig.gif)
- [Link GIF MaxPool](https://developers.google.com/machine-learning/practica/image-classification/images/maxpool_animation.gif)

**Implementação da Rede Neural**

In [ ]:
class Network_Mask_Detection_CNN(nn.Module):
    def __init__(self):
        super().__init__()        
        # convolutional layer (imagem de 32x32x3) 
        
        # convolutional layer (para 16x16x16)
        
        # convolutional layer (para 8x8x32)
        
        # linear layer (64 * 4 * 4 = 1024 para 16)
        
        # linear layer (16 para 1)
        
        # maxPooling para reduzir à metade cada imagem 
        self.pool = nn.MaxPool2d(2, 2)
        # dropout layer
        self.dropout = nn.Dropout(0.25)
        
        
    def forward(self, x):
        # Extração de Features usando Convolutional Layers e Maxpool
        
        # Transformar 64x4x4 para 1x1024
        x = x.view(-1, 64 * 4 * 4)
        
        # Dense Layers para Classificação
        # De 1024 para 16
        
        # De 16 para 1, com ativação sigmoidal para retornar um valor entre 0 e 1
        
        return x 

**Resumo da Rede Neural**

In [ ]:
# TODO - Criar a rede neural e mostrar um resumo dela
from torchsummary import summary


# 2 - Treinamento / Avaliação

In [ ]:
def train_epoch(model, trainLoader, optimizer, criterion):
    model.train()
    losses = []
    for X, y in trainLoader:
        y = y.type(torch.FloatTensor)
        optimizer.zero_grad()
        # (1) Passar os dados pela rede neural (forward)
        y_pred = model(X)              
        # (2) Calcular o erro da saída da rede com a classe das instâncias (loss)            
        loss = criterion(y_pred, y)        
        # (3) Usar o erro para calcular quanto cada peso (wi) contribuiu com esse erro (backward)
        loss.backward()
        # (4) Ataulizar os pesos da rede neural
        optimizer.step()        
        losses.append(loss.item())
        
    model.eval()
    return losses

def eval_model(model, loader):
    measures = []
    total = 0
    correct = 0
    for images, labels in loader:        
        y_pred = model(images).detach().numpy().round(0)        
        total += labels.size(0)
        correct += sum(y_pred == np.array(labels.unsqueeze(1).detach().numpy()))[0]        
    measures = {'acc' : correct/total}
    return measures

def train_and_evaluate(model, num_epochs, train_loader, test_loader):
  criterion = nn.BCELoss()  
  optimizer = optim.Adadelta(model.parameters())
  e_measures = []
  pbar = tqdm(range(num_epochs))
  for e in pbar:
      losses =  train_epoch(model, train_loader, optimizer, criterion)
      measures_on_train = eval_model(model, train_loader)
      measures_on_test  = eval_model(model, test_loader )
      train_loss = np.mean(losses)
      measures = {'epoch': e, 'train_loss': train_loss, 'train_acc' : measures_on_train['acc'].round(2), 'test_acc' : measures_on_test['acc'].round(2) }   
      pbar.set_postfix(measures)     
      e_measures += [measures]
  return pd.DataFrame(e_measures)   

In [ ]:
# TODO - Definir o número de epocas de treinamento

# TODO - Executar o treinamento da rede neural


# 3 - Avaliação: com Dados Reais

## Captura de Imagem da Webcam

Usar o Snippet do Google Colab para captura de imagem

In [ ]:
# TODO - Abrir a imagem salva e redimensionar


# TODO - Tranformar a imagem para o formato da rede


# TODO - Executar o modelo treinado

# A partir do valor entre 0 e 1 tomar a decisão
y_pred_class = np.round(y_pred)
y_pred_class_desc = label_desc[y_pred_class]

# Mostrar a Saída rede neural e a classe
print('Saída da rede neural:', y_pred)
print('Classe %i - %s' % (y_pred_class, y_pred_class_desc) )

# Considerações
- Conceito de classificação
- Conceitos de redes neurais: neurônio, função de ativação, arquitetura e treinamento
- Tipos de camadas:
  - Lineares ou Densas
  - Convolucionais e Maxpool
  - Dropout

## O que ficou de fora?
- Gradiente Descendente e Backpropagation (duas técnicas importantes para o ajuste dos pesos)
- Critério para a rede medir o erro
- Taxa de aprendizado usada no Optimizer ao longo do treinamento
- Usar Redes Neurais pré-treinadas para Extração de Features (vgg16, ResNet, Inception, ...)

## Disciplinas importantes para entender os fundamentos e implementação de Redes Neurais:
- Estatística
- Cálculo e Cálculo Numérico
- Linguagens de Programação
- Ciência de Dados ou Mineração de Dados

## Onde encontrar Cursos e Canais Gratuítos para Aprender Mais
- [Khan Academy](https://pt.khanacademy.org/math/statistics-probability)
- [Coursera](https://www.coursera.org/learn/neural-networks-deep-learning)
- [Kaggle Courses](https://www.kaggle.com/learn/overview)
- Youtube


